In [15]:
!pip3 install -q -r requirements.txt

In [1]:
from langame_client import LangameClient
from pprint import pprint
from firebase_admin import firestore

In [2]:
langame_client = LangameClient()

In [ ]:
# https://emojipedia.org/search
pprint(langame_client.generate_save_memes_general(topics_with_emojis = [
    ("knowledge", "🤔🧠💭📚📖"),
],
    memes_per_topic = 3,
    memes_per_meme=3,
    wikipedia_description = False,
    openai_description = False,
    anti_rate_limit_delay = 1))

In [4]:
for t in langame_client._firestore_client.collection_group(u"tags")\
    .where(u"topic.content", u"in", ["transhumanism"])\
    .order_by(u"createdAt", direction=firestore.Query.DESCENDING)\
    .limit(10)\
    .stream():
    pprint(t.to_dict())

In [ ]:
import json
memes = []
for m in langame_client._firestore_client.collection(u"memes").stream():
    v = m.to_dict()
    v['createdAt'] = v['createdAt'].timestamp()
    v['id'] = m.id
    v['tags'] = [] 
    for t in m.reference.collection('tags').stream():
        x = t.to_dict()
        x['createdAt'] = x['createdAt'].timestamp()
        x['id'] = t.id
        v['tags'].append(x)
    memes.append(v)

with open('memes.json', 'w') as outfile:
    json.dump({'memes': memes}, outfile, indent=4)


In [ ]:
import json
from datetime import datetime
with open('memes.json', 'r') as f:
    memes = json.load(f)
    pprint(memes)
    
    for m in memes.get('memes'):
        ref = langame_client._firestore_client.collection(u"memes")\
            .doc(m.get('id'))\
            .set({
                'content': m.get('content'),
                'createdAt': datetime.fromtimestamp(m.get('createdAt')),
            }).reference
        for t in memes.get('memes').get('tags'):
            ref.collection('tags').doc(t.get('id'))\
                .set({
                    'createdAt': datetime.fromtimestamp(t.get('createdAt')),
                    'topic': t.get('topic'),
                    'classification': t.get('classification'),
                    'origin': t.get('origin'),
                    'feedback': t.get('feedback'),
                    'context': t.get('context'),
                })
    


In [ ]:
for t in langame_client._firestore_client.collection_group(u"players")\
    .stream():
    
    #langame_client._firestore_client.doc(f"players")
        pprint(t.reference.delete())

In [ ]:
for t in langame_client._firestore_client.collection(u"langames")\
    .list_documents():
        pprint(t.delete())
        for p in t.collection("players").stream():
            pprint(p.reference.delete())

In [5]:
# TODO: use protobuf
p = {}
p["type"] = "TopicGeneralist"
p["template"] = """It is a question that leads to extremely profound conversations between people about "[TOPIC]":
1."""
t = {
    "createdAt": firestore.SERVER_TIMESTAMP,
    "engine": {
        "parameters": {
            "model": "davinci",
            "temperature": 0.7,
            "maxTokens": 150,
            "topP": 1,
            "frequencyPenalty": 0.1,
            "presencePenalty": 0.1,
            "stop": ["\\n", "2."],
        }
    }
}

In [6]:
p_col = langame_client._firestore_client.collection("prompts")
new_prompt_ref = p_col.add(p)
pprint(p_col.document(new_prompt_ref[1].id).collection("tags").add(t))

(DatetimeWithNanoseconds(2021, 7, 14, 9, 16, 13, 380577, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference object at 0x7f2028306730>)


In [7]:
langame_client.prompt_to_meme(topic_with_emoji = 
    ("transhumanism", "🤔🧠💭📚📖📐🧬🤖🦾🦿"),
)

calling openai with {'id': 'soMNZ7yJk3LPlYNzXLZd', 'prompt': 'It is a question that leads to extremely profound conversations between people about "transhumanism":\n1.', 'parameters': {'frequencyPenalty': 0.1, 'presencePenalty': 0.1, 'topP': 1, 'maxTokens': 150, 'model': 'davinci', 'temperature': 0.7, 'stop': ['\\n', '2.']}}


(DatetimeWithNanoseconds(2021, 7, 14, 9, 16, 37, 711119, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f1fce41fcd0>)

In [8]:
langame_client.prompt_to_meme(topic_with_emoji = 
    ("politic", "🤔💭📜🏛👨👩⚖🗽"),
)

calling openai with {'id': 'soMNZ7yJk3LPlYNzXLZd', 'prompt': 'It is a question that leads to extremely profound conversations between people about "politic":\n1.', 'parameters': {'maxTokens': 150, 'presencePenalty': 0.1, 'model': 'davinci', 'frequencyPenalty': 0.1, 'stop': ['\\n', '2.'], 'topP': 1, 'temperature': 0.7}}


(DatetimeWithNanoseconds(2021, 7, 14, 9, 16, 46, 637711, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f202835cc40>)

In [9]:
langame_client.prompt_to_meme(topic_with_emoji = 
    ("mind", "🤔🧠💭📚📖"),
)

calling openai with {'id': 'soMNZ7yJk3LPlYNzXLZd', 'prompt': 'It is a question that leads to extremely profound conversations between people about "mind":\n1.', 'parameters': {'temperature': 0.7, 'topP': 1, 'stop': ['\\n', '2.'], 'maxTokens': 150, 'model': 'davinci', 'frequencyPenalty': 0.1, 'presencePenalty': 0.1}}


(DatetimeWithNanoseconds(2021, 7, 14, 9, 16, 49, 805135, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f1fce41fe20>)

In [12]:
for e in langame_client._firestore_client.collection("memes").stream():
    for i in e.reference.collection("tags").where("topic.content", "==", "book").stream():
        pprint(i.to_dict())

{'createdAt': DatetimeWithNanoseconds(2021, 7, 6, 8, 54, 58, 798000, tzinfo=<UTC>),
 'topic': {'content': 'book', 'emojis': ['🤔', '🧠', '💭', '📚', '📖']}}
{'createdAt': DatetimeWithNanoseconds(2021, 7, 6, 8, 55, 25, 894000, tzinfo=<UTC>),
 'topic': {'content': 'book', 'emojis': ['🤔', '🧠', '💭', '📚', '📖']}}
{'createdAt': DatetimeWithNanoseconds(2021, 7, 7, 15, 25, 50, 420000, tzinfo=<UTC>),
 'topic': {'content': 'book', 'emojis': ['🤔', '🧠', '💭', '📚', '📖']}}
{'createdAt': DatetimeWithNanoseconds(2021, 7, 7, 15, 25, 56, 740000, tzinfo=<UTC>),
 'topic': {'content': 'book'}}
{'createdAt': DatetimeWithNanoseconds(2021, 7, 6, 8, 55, 25, 250000, tzinfo=<UTC>),
 'topic': {'content': 'book', 'emojis': ['🤔', '🧠', '💭', '📚', '📖']}}
{'createdAt': DatetimeWithNanoseconds(2021, 7, 6, 8, 57, 48, 777000, tzinfo=<UTC>),
 'topic': {'content': 'book', 'emojis': ['🤔', '🧠', '💭', '📚', '📖']}}
{'createdAt': DatetimeWithNanoseconds(2021, 7, 6, 8, 57, 48, 112000, tzinfo=<UTC>),
 'topic': {'content': 'book', 'emojis':

KeyboardInterrupt: 